In [8]:
pip install scikit-survival

     |████████████████████████████████| 2.5 MB 260 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 126 kB 456 kB/s eta 0:00:01
     |████████████████████████████████| 215 kB 752 kB/s eta 0:00:01
     |████████████████████████████████| 24.9 MB 132 kB/s eta 0:00:01
     |████████████████████████████████| 921 kB 801 kB/s eta 0:00:01
  Created wheel for scikit-survival: filename=scikit_survival-0.16.0-cp38-cp38-linux_x86_64.whl size=4892814 sha256=00ea5db5241931517ac26cb50867a28ba637f156c22cf648c9f0ba5596f83f23
  Stored in directory: /home/rashi/.cache/pip/wheels/de/63/7c/f9e1b7ab156e53a8e73205efe95e8399bb5a86070a8fe7202b
  Created wheel for ecos: filename=ecos-2.0.7.post1-cp38-cp38-linux_x86_64.whl size=217270 sha256=1de3791a7588e323bc9be489f0033fe9c46be72255cfe49d4b5e81bd06255410
  Stored in directory: /home/rashi/.cache/pip/wheels/c3/c1/ce/b551b0b3a40c99

In [1]:
from sksurv.datasets import load_gbsg2
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import ExtraSurvivalTrees
from sklearn.model_selection import train_test_split
from sksurv.preprocessing import OneHotEncoder
from sksurv.metrics import integrated_brier_score
import numpy

In [2]:
X, y = load_gbsg2()
X.loc[:, "tgrade"] = X.loc[:, "tgrade"].map(len).astype(int)
Xt = OneHotEncoder().fit_transform(X)

In [3]:
X.columns

Index(['age', 'estrec', 'horTh', 'menostat', 'pnodes', 'progrec', 'tgrade',
       'tsize'],
      dtype='object')

In [4]:
Xt.shape

(686, 8)

In [5]:
y.shape

(686,)

In [6]:
random_state = 20
X_train, X_test, y_train, y_test = train_test_split(
    Xt, y, test_size=0.25, random_state=random_state)
times = numpy.arange(365, 1826)

In [7]:
#1
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=random_state).fit(X_train, y_train)
survs = rsf.predict_survival_function(X_test)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])
score = integrated_brier_score(y_test, y_test, preds, times)
print(score)

0.17731976122800755


In [15]:
preds.shape

(172, 1461)

In [16]:
y_test.shape

(172,)

In [60]:
#2
est = CoxPHSurvivalAnalysis(ties="efron").fit(X_train, y_train)
survs = est.predict_survival_function(X_test)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])
score = integrated_brier_score(y_test, y_test, preds, times)
print(score)

0.1765894001540849


In [61]:
#3
estimator = GradientBoostingSurvivalAnalysis(loss="coxph").fit(X_train, y_train)
survs = estimator.predict_survival_function(X_test)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])
score = integrated_brier_score(y_test, y_test, preds, times)
print(score)

0.18754099548753692


In [64]:
#4
est = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True).fit(X_train, y_train) 
survs = est.predict_survival_function(X_test)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])
score = integrated_brier_score(y_test, y_test, preds, times)
print(score)

0.18387030486035455


In [66]:
#5
est = ExtraSurvivalTrees().fit(X_train, y_train) 
survs = est.predict_survival_function(X_test)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])
score = integrated_brier_score(y_test, y_test, preds, times)
print(score)

0.17802749470228862


In [69]:
#6
est = ComponentwiseGradientBoostingSurvivalAnalysis(loss="coxph").fit(X_train, y_train) 
survs = est.predict_survival_function(X_test)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])
score = integrated_brier_score(y_test, y_test, preds, times)
print(score)

0.18472486686898992
